# Продвинутое использование Qdrant для RAG

В этом блокноте мы продолжим изучение Qdrant и шаг за шагом построим минимальный pipeline Retrieval Augmented Generation (RAG).

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from sentence_transformers import SentenceTransformer
from transformers import pipeline

client = QdrantClient(':memory:')
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

## Шаг 1. Подготовка данных и индексирование

In [ ]:
texts = [
    "Qdrant это векторная база данных с открытым исходным кодом.",
    "Она обеспечивает быстрый поиск по большим наборам векторов.",
    "Qdrant отлично подходит для проектов с использованием нейронных сетей.",
    "Хранилище поддерживает фильтрацию по метаданным.",
    "RAG комбинирует поиск документов и генерацию ответа."
]
embeddings = encoder.encode(texts)
collection_name = 'rag_tutorial'
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=len(embeddings[0]), distance=Distance.COSINE)
)
points = [PointStruct(id=i, vector=vec, payload={'text': text}) for i, (vec, text) in enumerate(zip(embeddings, texts))]
client.upsert(collection_name=collection_name, points=points)

## Шаг 2. Поиск релевантных документов

In [ ]:
query = 'Для чего нужен Qdrant?'
query_vec = encoder.encode(query)
hits = client.search(collection_name=collection_name, query_vector=query_vec, limit=3)
hits

## Шаг 3. Генерация ответа на основе найденных документов

In [ ]:
context = '
'.join(hit.payload['text'] for hit in hits)
qa_prompt = f'Вопрос: {query}
Документы:
{context}
Ответ:'
text_generator = pipeline('text-generation', model='gpt2')
answer = text_generator(qa_prompt, max_new_tokens=50)[0]['generated_text']
print(answer)

## Шаг 4. Дополнительные возможности

В Qdrant можно использовать фильтры, настраивать параметры поиска и хранить произвольные метаданные для более сложных сценариев.